In [35]:
import json, pandas

In [36]:
BROKER_DICT = None
with open('broker.json', "r") as broker_json_file:
    BROKER_DICT = json.load(broker_json_file)

SYMBOLS = ["vale3","petr4", "bpac11", "ceab3", "abev3", "b3sa3", "embr3", "ggbr4", "tots3"]

def broker_ranking_by_ticker(trades): 
    pnl={}
    for i,trade in trades.iterrows():
        buyer = trade['buyer']
        seller = trade['seller']
        qty = trade['qty']
        price = trade['price']
        volume = price * qty
        agressor_buyer = 1 if trade['agressor'] == 'A' else 0
        agressor_seller = 1 if trade['agressor'] == 'V' else 0
        buyer_stats_from_dict = pnl.get(buyer)
        if buyer_stats_from_dict is None:
            stats = {}
            stats['net'] = volume
            stats['qty'] = qty
            stats['agressor'] = agressor_buyer
            pnl[buyer] = stats
        else:
            buyer_stats_from_dict['net'] = buyer_stats_from_dict['net'] + volume
            buyer_stats_from_dict['qty'] = buyer_stats_from_dict['qty'] + qty
            buyer_stats_from_dict['agressor'] = buyer_stats_from_dict['agressor'] + agressor_buyer

            pnl[buyer] = buyer_stats_from_dict

        seller_stats_from_dict = pnl.get(seller)
        if seller_stats_from_dict is None:
            stats = {}
            stats['net'] = -volume
            stats['qty'] = qty
            stats['agressor'] = agressor_seller
            pnl[seller] = stats
        else:
            seller_stats_from_dict['net'] = seller_stats_from_dict['net'] - volume
            seller_stats_from_dict['qty'] = seller_stats_from_dict['qty'] + qty
            seller_stats_from_dict['agressor'] = seller_stats_from_dict['agressor'] + agressor_seller

            pnl[seller] = seller_stats_from_dict

    statistics_array = []
    for idx in pnl:
        broker_name = [x for x in BROKER_DICT if x['playerMarketCode'] == str(idx)][1]['playerNameFull']
        statistics_array.append([idx, broker_name, pnl[idx]['net'], pnl[idx]['qty'], pnl[idx]['agressor']])
    statistics = pandas.DataFrame(statistics_array, columns=['BROKER_ID', 'BROKER_NAME', 'NET', 'QTY', 'AGRESSOR'])
    print(statistics.sort_values('QTY',ascending=False))

In [37]:
trades = pandas.read_csv('20210420_bbt.log.csv',sep=';')

In [39]:
for symbol in SYMBOLS:
    print("\n\n")
    print("\n\n=============================================================================")
    print("===================================", symbol.upper(),"===================================")
    print("=============================================================================")
    broker_ranking_by_ticker(trades[trades['symbol'] == symbol.upper()])



================================== VALE3 ==================================
    BROKER_ID                     BROKER_NAME         NET      QTY  AGRESSOR
14          8               UBS Brasil. CCTVM  18747882.0  2765200      2301
1           3      XP INVESTIMENTOS CCTVM S.A -52130499.0  2330000      2925
28         13         MERRILL LYNCH S.A. CTVM -25250738.0  2119500       614
4         120                   BRASIL PLURAL  -1282870.0  1462400      1454
8        1618                           IDEAL   9834157.0  1289700      1266
12         45  CREDIT SUISSE BRASIL S.A. CTVM  91518926.0  1216700      4122
2          85            BTG PACTUAL CTVM S.A  41743156.0  1074500      1004
6          40  MORGAN STANLEY D. W. CTVM S.A. -32823650.0   971200       859
24         77                       CITIGROUP -55768921.0   865100       693
5          72              BRADESCO S.A. CTVM -10668241.0   630900       634
10        114                     ITAU CV S/A -25425228.0   542300      124

    BROKER_ID                     BROKER_NAME        NET      QTY  AGRESSOR
3           8               UBS Brasil. CCTVM -2905233.0  1531300      2231
11        114                     ITAU CV S/A -5963260.0  1140800       358
1           3      XP INVESTIMENTOS CCTVM S.A  6295870.0   845700       726
6          16          J. P. MORGAN CCVM S.A. -4844292.0   556300       694
8          40  MORGAN STANLEY D. W. CTVM S.A.  3728783.0   393300       357
2         308                 CLEAR CTVM LTDA  2050599.0   350100       811
0         120                   BRASIL PLURAL   835876.0   295400       207
17         85            BTG PACTUAL CTVM S.A   591374.0   282100       254
23       1618                           IDEAL   375236.0   267800       428
4         127  TULLETT PREBON BRASIL S.A. CVC -3073082.0   264400       404
42         13         MERRILL LYNCH S.A. CTVM -3098565.0   255200       100
20        262   MIRAE ASSET SECURITIES BRASIL   373282.0   151200       224
10       109

    BROKER_ID                     BROKER_NAME         NET      QTY  AGRESSOR
19         13         MERRILL LYNCH S.A. CTVM   8397150.0  7058800       453
2           8               UBS Brasil. CCTVM -12933194.0  6251000      5170
3          40  MORGAN STANLEY D. W. CTVM S.A. -15394513.0  2727600      2985
4        1618                           IDEAL   2682495.0  1664900      1712
8           3      XP INVESTIMENTOS CCTVM S.A   2519104.0  1479900      1368
1         120                   BRASIL PLURAL   1455452.0  1093200       175
7         308                 CLEAR CTVM LTDA   1635395.0   894800      1468
22        114                     ITAU CV S/A   6140189.0   662200       590
17       1982                      MODAL DTVM  -6490122.0   587400       918
5          72              BRADESCO S.A. CTVM   4575588.0   545900       606
13        386                   OCTO CTVM S/A   -375204.0   488400       522
6          85            BTG PACTUAL CTVM S.A    646710.0   449000       326